<a href="https://colab.research.google.com/github/tumblingdice512/Research/blob/master/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
import numpy as np
import random

In [2]:
#モデルについて
#幅N、深さL+1(l=0,1,･･･,L)のネットワーク
#各パーセプトロンはM成分を持つスピンで各成分は+1か-1を取る
#第l層と第l+1層の各パーセプトロンの同一成分は全て結合→重みJ
#各層の重みJはΣJ**2 = Nと正規化されている
#第l層のスピンの値に重みをかけた値に対して符号関数をかませたものが第l+1層の値(0に対しては+1か-1にする)
#初期条件としては、入力層と出力層のスピンの値を与える
#入力層のスピンの値と、初期の重みの条件から、出力層を予想して、
#
#
#
#

In [3]:
N = 20
L = 5

In [4]:
#def sgn(x):
#  if x != 0:
#    return np.sign(x)
#
#  else:
#    return 1
#不要

In [5]:
#print(sgn(-5))
#print(sgn(0))
#print(sgn(3))


-1
1
1


ValueError: ignored

In [ ]:
a = [1, -1]
b = random.choice(a)

print(b)

-1


In [ ]:
S0 = np.array([random.choice([1,-1]) for i in range(N)])
SL = np.array([random.choice([1,-1]) for i in range(N)])

print(S0)
print("S0の要素数は",len(S0))

print(SL)
print("SLの要素数は",len(SL))

#初期条件として与えるスピンを生成
#各パーセプトロンのスピンの成分数を増やすときは、SOとSLを行列の形式にすればよい。

[ 1  1 -1  1 -1 -1  1  1  1  1  1 -1 -1 -1  1  1 -1 -1  1 -1]
S0の要素数は 20
[ 1 -1  1 -1  1  1  1  1 -1  1 -1  1 -1  1  1  1 -1  1 -1  1]
SLの要素数は 20


In [ ]:
#S0 = np.array([[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]])
#SL = np.array([[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]])

#print(S0)

#print(SL)

#S0とSLの成分数が2の時

[[-1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1 -1  1  1 -1  1]
 [ 1  1  1  1  1 -1 -1  1  1  1  1 -1  1  1 -1  1 -1  1  1 -1]]
[[ 1 -1  1 -1 -1 -1  1  1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1]
 [ 1 -1 -1 -1 -1  1  1 -1  1  1  1  1  1  1 -1 -1 -1 -1 -1  1]]


In [ ]:
class Network():
  #ネットワークを定義する
  #構成[入力層, 第1層, 第2層, 第3層, 第4層, 第5層]
  #全結合

  def __init__(self):
    #重みの定義
    self.w1 = np.random.randn(1,N)
    self.w2 = np.random.randn(1,N)
    self.w3 = np.random.randn(1,N)
    self.w4 = np.random.randn(1,N)
    self.w5 = np.random.randn(1,N)

  def forward(self, x):
    #重みの正規化
    self.J1 = self.w1 * (np.abs(self.w1 + 1e-7))**(-1) * N
    self.J2 = self.w2 * (np.abs(self.w2 + 1e-7))**(-1) * N
    self.J3 = self.w3 * (np.abs(self.w3 + 1e-7))**(-1) * N
    self.J4 = self.w4 * (np.abs(self.w4 + 1e-7))**(-1) * N
    self.J5 = self.w5 * (np.abs(self.w5 + 1e-7))**(-1) * N

    self.layer0 = x
    self.layer1 = np.sign((np.dot(self.layer0, self.J1))+1e-7)
    self.layer2 = np.sign((np.dot(self.layer1, self.J2))+1e-7)
    self.layer3 = np.sign((np.dot(self.layer2, self.J3))+1e-7)
    self.layer4 = np.sign((np.dot(self.layer3, self.J4))+1e-7)
    self.layer5 = np.sign((np.dot(self.layer4, self.J5))+1e-7)

  def backward(self, t, y):
    #誤差逆伝播
    delta5 = -2*(t-y)
    delta4 = np.dot(delta5,self.J5.T)
    delta3 = np.dot(delta4,self.J4.T) 
    delta2 = np.dot(delta3,self.J3.T)
    delta1 = np.dot(delta2,self.J2.T)

    #重みの勾配
    self.dedJ5 = np.dot(self.layer4.T, delta5) / delta5.shape[0]
    self.dedJ4 = np.dot(self.layer3.T, delta4) / delta4.shape[0]
    self.dedJ3 = np.dot(self.layer2.T, delta3) / delta3.shape[0]
    self.dedJ2 = np.dot(self.layer1.T, delta2) / delta2.shape[0]
    self.dedJ1 = np.dot(self.layer0.T, delta1) / delta1.shape[0]

  def optimize_GradientDecent(self, lr):
        
      self.J1 -= lr * self.dedJ1
      self.J2 -= lr * self.dedJ2
      self.J3 -= lr * self.dedJ3
      self.J4 -= lr * self.dedJ4
      self.J5 -= lr * self.dedJ5

